In [1]:
import numpy as np
import pandas as pd
import os
import random

from lxml import etree
import urllib.request
import urllib.parse
import pandas as pd
import unicodedata

import re
import requests

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import seaborn as sns
import string
import nltk

###nltk.download('stopwords')
###nltk.download('punkt')
###nltk.download('averaged_perceptron_tagger')
###nltk.download('wordnet')
###nltk.download('omw-1.4')
###nltk.download('maxent_ne_chunker')
###nltk.download('words')"

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk.tag import pos_tag_sents
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize, pos_tag

## 1. read wta player styles

In [3]:
### read

path = '/Users/liuyiwu/540/project/player_style_male_and_female/player_style_female/'
name = []
data = []

for file_name in os.listdir(path):
    if file_name == '.DS_Store':
        continue
        
    f = open(path + file_name, encoding = 'utf-8')
    a = []
    content = f.readlines()
    for line in content:
        a.append(line.replace("\n", ""))
        f.close()
    name.append(file_name)
    data.append(''.join(a))

playing_style = data 

In [4]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')

### new stopwords selected based on frequency in the corpus
new_stopwords = ['serv','play','court','game','also','player','style','use','ball',
                 'allow','tour','describ','although','time','howev','wta','atp','note',
                 'consid','career','finish','year','need','prefer','said','take','number',
                 'employ','come','despit','skill','set','lot','record','result','could','comment',
                 'except','made','includ','sinc','per','somtim','amount','develop','extrem','even',
                 'bank','work','australian','feel','regard','favorit','world','among','much','instead',
                 'hour','em','whose','think','later','put','would','women','help','though','way','american',
                 'interview','alway','ever','look','she','almost','peopl','server','place','refer','problem',
                 'either','rather','keep','away','still','enough','someth','thing','john','kvitová','toward',
                 'see','util','build','began','level','unlike','mayb','solid','template', 'citation', 
                 'percentage', 'surface', 'us', 'french', 'open', 'serena', 'weakness', 'venus', 'wimbledon']

for i in new_stopwords:
    stopwords.append(i)


## 2. tokenization and stemming

In [5]:


from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string
    #将所有stopwords(无意义)的词语删除
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    #将所有数字删除
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [6]:
print(tokenization_and_stemming(playing_style[0])[0:20])

['dolehid', 'aggress', 'baselin', 'known', 'strong', 'serv', 'power', 'groundstrok', 'use', 'hit', 'high', 'winner', 'forehand', 'particular', 'one', 'best', 'shot', 'alreadi', 'advanc', 'teenag']


## 3. tokenization and lemmatization

In [7]:


# 获取单词的词性
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def tokenization_and_lemmatization(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string
    #将所有stopwords(无意义)的词语删除
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    #将所有数字删除
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
    
    tags = pos_tag(filtered_tokens) # 获取单词词性
    wnl = WordNetLemmatizer()
    lemmatization = []
    
    #lemmatization
    for tag in tags:
        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
        lemmatization.append(wnl.lemmatize(tag[0], pos=wordnet_pos)) # 词形还原
    
    return lemmatization

In [8]:
tokenization_and_lemmatization(playing_style[0])[0:20]

['dolehide',
 'aggressive',
 'baseliner',
 'know',
 'strong',
 'serve',
 'powerful',
 'groundstrokes',
 'us',
 'hit',
 'high',
 'winner',
 'forehand',
 'particular',
 'one',
 'best',
 'shot',
 'already',
 'advanced',
 'teenager']

In [9]:
playing_style[0]

'Dolehide is an aggressive baseliner.[33] She is known for having a strong serve and powerful groundstrokes, which she uses to a hit a high number of winners.[25][26] Her forehand in particular is one of her best shots and was already very advanced while she was still a teenager.[15] CiCi Bellis faced Dolehide at the 2014 Orange Bowl when both players were still juniors and commented that Dolehide "hits probably the hardest by far" compared to Bellis\'s other opponents and said "her serve is amazing."[10] Venus Williams defeated Dolehide at the 2018 Canadian Open, but commented that "she had a really great second serve."[34]'

## 4. tf-idf

In [10]:
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram

#tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
#                                 min_df=0.01, stop_words='english',
#                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(2,3))

tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_lemmatization, ngram_range=(2,3))

tfidf_matrix = tfidf_model.fit_transform(playing_style) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      "player style txt and " + str(tfidf_matrix.shape[1]) + " terms.")

In total, there are 103player style txt and 1000 terms.


In [11]:
# words
tf_selected_words = tfidf_model.get_feature_names()
tf_selected_words[0:100]

/Users/liuyiwu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['ability hit',
 'ability slide',
 'able hit',
 'able hit winner',
 'accumulates significant',
 'accumulates significant number',
 'accurate groundstrokes',
 'accurate serve',
 'ace count',
 'ace dictate',
 'ace dictate stroke',
 'ace frequently',
 'ace match',
 'ace serve',
 'add reliable',
 'add reliable source',
 'adept hit',
 'adept hit backhand',
 'adept net',
 'aggression score',
 'aggressive baseline',
 'aggressive baseliner',
 'aggressive baseliner center',
 'aggressive mindset',
 'aggressive play',
 'aggressive player',
 'aggressive playing',
 'aggressive return',
 'aggressive return serve',
 'aggressive shot',
 'agnieszka radwańska',
 'aid point',
 'aid point construction',
 'aim point',
 'allow dictate',
 'allow dominate',
 'allow execute',
 'allow generate',
 'allow hit',
 'allow hit winner',
 'allow serve',
 'allow serve ace',
 'allow serve multiple',
 'allow serve numerous',
 'angelique kerber',
 'angle forehand',
 'angle forehand backhand',
 'apply slice',
 'apply slice 

## LDA

In [90]:
random.seed(46)
# Use LDA for clustering， 7 topics （7 classes of players), tried 8 topics 2 of which are highly similar 
# to each other
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=4, max_iter=10)

In [91]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(103, 4)
[[0.04772266 0.85453048 0.04976674 0.04798011]
 [0.02314252 0.0284118  0.92534888 0.02309681]
 [0.02424711 0.92753317 0.02417146 0.02404826]
 [0.01832626 0.94496344 0.01840447 0.01830583]
 [0.03332912 0.33882153 0.5958206  0.03202875]
 [0.08449208 0.0866183  0.74277829 0.08611133]
 [0.05096612 0.84754504 0.0507183  0.05077054]
 [0.03096579 0.43518749 0.50234109 0.03150562]
 [0.0225192  0.93230878 0.02280357 0.02236845]
 [0.8607786  0.04799141 0.04587847 0.04535152]
 [0.06677332 0.06970195 0.06699843 0.7965263 ]
 [0.05503192 0.05600472 0.05550663 0.83345673]
 [0.12509904 0.12723782 0.12716241 0.62050073]
 [0.68562492 0.10649279 0.10418343 0.10369887]
 [0.06144094 0.06388098 0.81292731 0.06175077]
 [0.04701713 0.0522004  0.85398092 0.04680156]
 [0.79543964 0.07019192 0.06685914 0.0675093 ]
 [0.02015432 0.93955416 0.02027746 0.02001406]
 [0.01950245 0.94163873 0.01949083 0.01936799]
 [0.03428553 0.2140449  0.71796146 0.03370811]
 [0.05717938 0.06064293 0.8245717  0.05760599]
 [0.

In [52]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(7, 1000)
[[0.14285769 0.14285827 0.56265655 ... 0.34572279 0.34572279 0.46924618]
 [0.14285782 0.23892403 0.14285803 ... 0.14295117 0.14295117 0.14285807]
 [0.14285783 0.14285857 0.14304798 ... 0.14290226 0.14290226 0.14285807]
 ...
 [0.14325037 0.34313581 0.14285795 ... 0.43426677 0.43426677 0.14285798]
 [0.28132811 0.23591786 0.39064477 ... 0.6951197  0.6951197  0.38868884]
 [0.53915536 0.14982252 0.61220423 ... 0.96407009 0.96407009 0.14285782]]


In [53]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
name1 = [ " ".join(n.split(".")[:-1]) for n in name]
doc_names = [name1[i] for i in range(len(playing_style))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic


df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,topic
Caroline Dolehide,0.83,0.03,0.03,0.03,0.03,0.03,0.03,0
Donna Vekic,0.76,0.01,0.01,0.01,0.01,0.01,0.18,0
Caroline Wozniacki,0.01,0.01,0.01,0.01,0.01,0.01,0.91,6
Barbora Krejcikova,0.53,0.01,0.01,0.01,0.01,0.01,0.42,0
Kristina Mladenovic,0.02,0.02,0.02,0.02,0.02,0.02,0.89,6
Yaroslava Shvedova,0.05,0.05,0.71,0.05,0.05,0.05,0.05,2
Coco Vandeweghe,0.03,0.03,0.03,0.83,0.03,0.03,0.03,3
Petra Kvitova,0.02,0.02,0.02,0.02,0.89,0.02,0.02,4
Elise Mertens,0.01,0.01,0.01,0.01,0.92,0.01,0.01,4
Anastasija Sevastova,0.86,0.02,0.02,0.02,0.02,0.02,0.02,0


In [32]:
df_document_topic.to_csv("/Users/liuyiwu/540/project/f_document_topic.csv")

In [33]:
df_document_topic['topic'].value_counts().to_frame()

,topic
18,15
6,7
11,6
10,4
9,4
4,4
12,4
29,4
7,4
2,4


In [54]:
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names

df_topic_words = df_topic_words.round(2)
df_topic_words

/Users/liuyiwu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,ability hit,ability slide,able hit,able hit winner,accumulates significant,accumulates significant number,accurate groundstrokes,accurate serve,ace count,ace dictate,...,winner point,winner position,winner position forehand,winner possess,winner shoot,winner shot,winner typically,winner unforced,winner unforced error,work early
Topic0,0.14,0.14,0.56,0.57,0.27,0.27,0.36,0.34,0.38,0.27,...,0.36,0.46,0.14,0.14,0.14,0.29,0.28,0.35,0.35,0.47
Topic1,0.14,0.24,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,...,0.14,0.34,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14
Topic2,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.26,0.14,0.14,...,0.14,0.23,0.14,0.14,0.27,0.14,0.14,0.14,0.14,0.14
Topic3,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14
Topic4,0.14,0.34,0.14,0.14,0.14,0.14,0.14,0.33,0.24,0.23,...,0.14,0.22,0.14,0.29,0.14,0.15,0.14,0.43,0.43,0.14
Topic5,0.28,0.24,0.39,0.14,0.14,0.14,0.68,0.39,0.31,0.14,...,0.28,0.35,0.24,0.14,0.24,0.31,0.32,0.70,0.70,0.39
Topic6,0.54,0.15,0.61,0.28,0.35,0.35,0.14,0.14,0.23,0.66,...,0.14,0.39,0.28,0.44,0.14,0.38,0.14,0.96,0.96,0.14


here we get the 

In [92]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=20)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,play style,forehand hit,drop shot,backhand forehand,hit ball,return game,big asset,win point,monica seles,style play,shot play,hard court,main weakness,favorite surface,accurate groundstrokes,break opponent,backhand line,percentage return,point return,play play
Topic 1,hit winner,second serve,unforced error,forehand backhand,double fault,powerful groundstrokes,wta tour,style play,backhand slice,hard court,grass court,playing style,serve ace,drop shot,allow serve,french open,mph allow,sharp angle,mph allow serve,winner unforced
Topic 2,solid solid,style play,solid solid solid,high number,drop shot,powerful forehand use,second round,unforced error,use forehand,produce high number,aggressive style play,produce high,serve forehand,u open,powerful serve,aggressive style,powerful forehand,play match,high number winner,long rally
Topic 3,play backhand,speed court,match wimbledon,topspin forehand,surface clay,primary strength,note powerful,game great,female player,prefer surface,posse high topspin,posse high,serve able,know fast,clay surface,justine henin,shot backhand,title clay,powerful forehand,return serf


In [36]:
### classify players into 8 style groups based on 5 types of features
### 1. Aggresive/Defensive/natural  2.Court preference (4 types)   3. Backhand/fronthand/both  4.hit hard/weak
style_dic = {}
style_dic["Topic0"] = ["balanced", "no", "backhand", "powerful"]
style_dic["Topic1"] = ["balanced", "hard", "both", "powerful"]
style_dic["Topic2"] = ["aggressive", "grass or hard", "both", "weak"]
style_dic["Topic3"] = ["aggresive", "no", "both", "hard"]
style_dic["Topic4"] = ["aggresive", "grass", "both", "hard"]
### similar features "approach net" and "aggresive"
style_dic["Topic5"] = ["balanced", "no", "both", "hard"]
### waiting for opportunity = defensive
style_dic["Topic6"] = ["defensive", "grass and clay", "backhand", "hard"]

In [37]:

# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 7

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [38]:
product = {'players': doc_names, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['players', 'cluster'])

In [39]:
frame.head(10)

,players,cluster
0,Caroline Dolehide,5
1,Donna Vekic,2
2,Caroline Wozniacki,5
3,Barbora Krejcikova,1
4,Kristina Mladenovic,3
5,Yaroslava Shvedova,5
6,Coco Vandeweghe,5
7,Petra Kvitova,6
8,Elise Mertens,0
9,Anastasija Sevastova,4


In [40]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
1,29
0,22
5,19
3,11
4,11
6,6
2,5


In [41]:
#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :15]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print()
  
    cluster_reviews = frame[frame.cluster==i].players.tolist()
    print ("Cluster " + str(i) + " players (" + str(len(cluster_reviews)) + " players): ")
    #print (", ".join(cluster_reviews))
    print()

Cluster 0 words:backhand slice,unforced error,hit winner,allow serve,peak mph,average mph,mph allow,second serve,serve ace,mph allow serve,hit backhand,double experience,posse effective,hard court,forehand backhand,
Cluster 0 players (22 players): 

Cluster 1 words:double fault,unforced error,hit winner,second serve,allow hit,serve abbreviate,winner unforced,winner unforced error,serve lack,offensive baseliner,powerful forehand,win match,long rally,backhand forehand,aggressive playing,
Cluster 1 players (29 players): 

Cluster 2 words:forehand hit,center border,border center border,border center,remove message,learn remove message,learn remove,hidden collapse border,medium section,center border important,center border center,border important medium,border important,fbfbfb html,collapse border border,
Cluster 2 players (5 players): 

Cluster 3 words:drop shot,slice drop shot,slice drop,backhand line,hit slice,strength serve,shot develop,play clay,hard hit,big asset,slice forehand,shot b

In [47]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')

### new stopwords selected based on frequency in the corpus
new_stopwords = ['serv','play','court','game','also','player','style','use','ball',
                 'allow','tour','describ','although','time','howev','wta','atp','note',
                 'consid','career','finish','year','need','prefer','said','take','number',
                 'employ','come','despit','skill','set','lot','record','result','could','comment',
                 'except','made','includ','sinc','per','somtim','amount','develop','extrem','even',
                 'bank','work','australian','feel','regard','favorit','world','among','much','instead',
                 'hour','em','whose','think','later','put','would','women','help','though','way','american',
                 'interview','alway','ever','look','she','almost','peopl','server','place','refer','problem',
                 'either','rather','keep','away','still','enough','someth','thing','john','kvitová','toward',
                 'see','util','build','began','level','unlike','mayb','solid','template', 'citation', 
                 'percentage', 'surface', 'us', 'french', 'open', 'serena', 'weakness', 'venus', 'wimbledon']

for i in new_stopwords:
    stopwords.append(i)

In [56]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')

In [74]:

tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_lemmatization, ngram_range=(2,3))

tfidf_matrix = tfidf_model.fit_transform(playing_style) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      "player style txt and " + str(tfidf_matrix.shape[1]) + " terms.")

In total, there are 103player style txt and 1000 terms.


In [75]:
tf_selected_words = tfidf_model.get_feature_names()

X = tfidf_model.fit_transform(playing_style)

top_n = 50
print('tf_idf scores: \n', sorted(list(zip(tfidf_model.get_feature_names(), 
                                             X.sum(0).getA1())), 
                                 key=lambda x: x[1], reverse=True)[:top_n])

tf_idf scores: 
 [('style play', 3.9673124945590317), ('unforced error', 3.4270441250778974), ('hit winner', 3.361322200283284), ('drop shot', 3.217677534671646), ('forehand backhand', 3.2085044742828264), ('second serve', 3.0373871094395013), ('play backhand', 2.8638572882871047), ('double fault', 2.5801053069663866), ('hit ball', 2.4273950510334377), ('hard court', 2.387781236108507), ('backhand slice', 2.1290214289715887), ('wta tour', 2.088563278448431), ('playing style', 2.0784453947955797), ('aggressive style', 2.065910919834918), ('play style', 1.9875569752513937), ('powerful groundstrokes', 1.9763381661142436), ('finish point', 1.9756870953190124), ('favourite surface', 1.928883695919309), ('french open', 1.9244568988506898), ('forehand hit', 1.924233688687468), ('powerful serve', 1.8841741911408858), ('grass court', 1.849153478384462), ('powerful forehand', 1.8345176521111484), ('hit flat', 1.8209713410996486), ('grand slam', 1.7565203488539984), ('number winner', 1.7241202761

In [79]:
lda = LatentDirichletAllocation(n_components=7, max_iter=10)
lda_output = lda.fit_transform(tfidf_matrix)

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=20)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

/Users/liuyiwu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,double fault,hit winner,second serve,forehand backhand,serve ace,unforced error,allow serve,mph allow,backhand slice,mph allow serve,peak mph,playing style,winner unforced,winner unforced error,average mph,double experience,wta tour,allow serve ace,create opportunity,drop shot
Topic 1,solid solid,play backhand,solid solid solid,shot backhand,powerful serve groundstrokes,clay court,unforced error,net play,flat groundstrokes,aggressive baseliner,female player,groundstrokes hit,volley skill,favourite shot,hit shot,hit powerful,serve major,note powerful,center border,border center border
Topic 2,clean winner,monica seles,french open,service game,backhand forehand,shot forehand,play backhand,create sharp,create sharp angle,main weakness,come net,style play,martina hingis,capable hit,nigel sears,offensive baseliner,sharp angle,game base,martina navratilova,tennis player
Topic 3,style play,hit slice,aggressive style play,drop shot,aggressive style,best surface,slice drop,slice drop shot,strength serve,petra kvitová,hard court,play clay,main weakness,backhand line,shot hit,lot ball,ball play,aggressive player,point player,court title
Topic 4,forehand hit,play style,drop shot,play surface,big asset,enjoy play,player use,baseline forehand,surface like,shot play,martina hingis,backhand line,favourite shot,style play,tennis like,prepare hit,break opponent,slice drop,slice drop shot,play tennis
Topic 5,hit ball,style play,match wimbledon,note powerful,hard court,volley winner,play clay,grass hard court,aggressive style,really really,play backhand,play grass,clay court,grass hard,speed court,forehand backhand,backhand solid,primary strength,surface grass,kim clijsters
Topic 6,mile hour,unforced error,allow hit,hit large,hit large number,number winner,strong forehand,serena williams,u open,large number winner,finish point,style play,win point,aggression score,grand slam,hard court,backhand serve,game style,rarely win,speed court


In [87]:
import tmtoolkit
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim
def topic_model_coherence_generator(topic_num_start=2,
                                    topic_num_end=10,
                                    norm_corpus='',
                                    cv_matrix='',
                                    cv=''):
    norm_corpus_tokens = [doc.split() for doc in norm_corpus]
    models = []
    coherence_scores = []

    for i in range(topic_num_start, topic_num_end):
        print(i)
        cur_lda = LatentDirichletAllocation(n_components=i,
                                            max_iter=10000,
                                            random_state=0)
        cur_lda.fit_transform(cv_matrix)
        cur_coherence_score = metric_coherence_gensim(
            measure='c_v',
            top_n=20,
            topic_word_distrib=cur_lda.components_,
            dtm=cv.fit_transform(norm_corpus),
            vocab=np.array(cv.get_feature_names()),
            texts=norm_corpus_tokens)
        models.append(cur_lda)
        coherence_scores.append(np.mean(cur_coherence_score))
    return models, coherence_scores

In [88]:
models, coherence_scores = topic_model_coherence_generator(
    2, 10, norm_corpus=playing_style, cv=tfidf_model, cv_matrix=tfidf_matrix)

2


/Users/liuyiwu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/liuyiwu/opt/anaconda3/lib/python3.9/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:202: RuntimeWarning: invalid value encountered in true_divide
  numerator = (co_occur_count / num_docs) + EPSILON
/Users/liuyiwu/opt/anaconda3/lib/python3.9/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:203: RuntimeWarning: invalid value encountered in true_divide
  denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
/Users/liuyiwu/opt/anaconda3/lib/python3.9/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:198: RuntimeWarning: invalid value encountered in true_divide
  co_doc_prob = co_occur_count / num_docs


3
4
5
6
7
8
9


In [89]:
coherence_scores

[nan, nan, nan, nan, nan, nan, nan, nan]

In [71]:
tfidf_model1 = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_lemmatization)

tfidf_matrix1 = tfidf_model1.fit_transform(playing_style) #fit the vectorizer to synopses

tf_selected_words1 = tfidf_model1.get_feature_names()

top_n = 50
print('tf_idf scores: \n', sorted(list(zip(tfidf_model1.get_feature_names(), 
                                             tfidf_matrix1.sum(0).getA1())), 
                                 key=lambda x: x[1], reverse=True)[:top_n])

tf_idf scores: 
 [('serve', 9.800798410725022), ('hit', 8.772963184616527), ('play', 7.724640209730993), ('backhand', 7.125316325278012), ('shot', 7.004139243452403), ('forehand', 6.606944987566251), ('court', 6.425613702424704), ('game', 5.9874474789394565), ('winner', 5.741307356214546), ('player', 5.306744007768669), ('point', 5.23147102461234), ('opponent', 4.838503770789364), ('allow', 4.806933249995407), ('style', 4.686196673599501), ('powerful', 4.2951030142223985), ('surface', 4.245227421544899), ('aggressive', 4.220549115624489), ('strong', 4.125513123184971), ('groundstrokes', 3.9263384990454027), ('net', 3.8181619815670427), ('use', 3.796738125566304), ('ball', 3.794929280676019), ('return', 3.701067568416116), ('match', 3.651519795781254), ('power', 3.583114396839136), ('double', 3.5028298488931338), ('know', 3.432142212901573), ('hard', 3.3004223633813075), ('clay', 3.2698650177108854), ('slice', 3.2364250689916543), ('second', 3.097174026771613), ('grass', 3.0660898291374